In [ ]:
#Import Custom Tools
import ldasort.ldasort as ldasort

## Import Standard Data Processing Tools
import numpy as np
import matplotlib.pyplot as plt

#import ML tools
import tensorflow as tf
from tensorflow import keras

In [ ]:
#LOAD FASHION MNIST
from tensorflow.keras.datasets import mnist

data_set = "MNIST"

(x_train, y_train), (x_test, y_test) = mnist.load_data()

#normalize
x_train_normalized = (x_train - np.mean(x_train))/np.std(x_train) 
x_test_normalized = (x_test - np.mean(x_train))/np.std(x_train)

#flatten and reshape
x_train_normalized = x_train_normalized.reshape(60000,28*28)
x_test_normalized = x_test_normalized.reshape(10000,28*28)
y_train=np.array(y_train)


In [ ]:
# Find weights and biases using LDA sorting algorithm. 
# Comment out if weights already present. 

weights, biases = ldasort.ldasort(x_train_normalized, y_train, verbose=True)

In [ ]:
# Save weights and biases for use later
np.savez("Sorted_Weights_{}".format(dataset), weights=weights, biases=biases)

In [ ]:
### create loop for comparing across batch sizes, learning rates

learning_rates = [.001, .005, .01]
batch_sizes = [25, 100, 500]

#small values for quick sample run 
n_trials = 5
n_epochs = 5

t0=timer()

for batch_size in batch_sizes:
    for lr in learning_rates:

        temp_hists=[]
        val_hists=[]

        for trial in range(n_trials):

            #set up LDA-initialized model
            
            model=setup(factors=len(all_weights),input_shape=784, output_shape=10)

            model.compile(
                    optimizer=keras.optimizers.SGD(lr=lr),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'],
                    weighted_metrics=['accuracy']
                    )

            model.layers[0].set_weights([all_weights.T,all_biases])
            
            t0loss, t0acc, t0wacc = model.evaluate(X,y_train,verbose=False)


            history=model.fit(X, 
                      y_train, 
                      epochs=n_epochs, 
                      batch_size=batch_size,
                      validation_data = (x_test_normalized, y_test),
                      verbose=False)

            temp_hists.append([t0acc]+history.history['acc'])
            print("LDA trial {} Done, time elapsed {} minutes".format(trial, np.round((timer()-t0)/60),3))

            keras.backend.clear_session()

        lda_hist = temp_hists
        val_lda_hist=val_hists


        #######################################
        #Run Trials with Random Initialization#
        #######################################
        temp_hists=[]
        val_hists=[]

        for trial in range(n_trials):


            model=setup(factors=len(all_weights),input_shape=784, output_shape=10)

            model.compile(
                    optimizer=keras.optimizers.SGD(lr=lr),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'],
                    weighted_metrics=['accuracy']
                    )
            
            t0loss, t0acc, t0wacc = model.evaluate(X,y_train,verbose=False)


            history=model.fit(X, 
                      y_train, 
                      epochs=n_epochs, 
                      batch_size=batch_size,
                      validation_data = (x_test_normalized, y_test),
                      verbose=False)

            temp_hists.append([t0acc]+history.history['acc'])
            print("Random trial {} Done, time elapsed {} minutes".format(trial, np.round((timer()-t0)/60)),3)

            keras.backend.clear_session()

        rand_hist = temp_hists
        val_rand_hist=val_hists


        np.savez("{}_sigmoid_lr{}_bat{}".format(data_set, lr,batch_size), 
                 lda_acc=lda_hist, 
                 val_lda_acc=val_lda_hist,
                 rand_acc = rand_hist, 
                 val_rand_acc = val_rand_hist,
                 n_components=len(all_weights), 
                num_trials=n_trials,
                n_epochs = n_epochs)
